In [32]:
import psycopg2
import pandas as pd
import sys
from datetime import datetime

In [33]:
try:
    conn = psycopg2.connect(
        host=input('Enter host name: '),
        port=eval(input('Enter port: ')),
        database=input('Enter database name: '),
        user=input('Enter username: '),
        password=input('Enter password: ')
    )
    print('Connected to database successfully...')
except Exception as e:
    print(f'Connection error: {e}')

Enter host name: localhost
Enter port: 5432
Enter database name: healthcare_data
Enter username: postgres
Enter password: Hewasborn@1889
Connected to database successfully...


In [34]:
try:
    conn.autocommit=True
    print('auto commit set to 1...')
except Exception as e:
    print(f'{e}')

auto commit set to 1...


In [16]:
def create_schema():
    try:
        cur = conn.cursor()
        cur.execute("CREATE SCHEMA IF NOT EXISTS healthcare;")
        conn.commit()
        print('Schema created...')
    except Exception as e:
        print(f'Schema creation error: {e}')
        conn.rollback()
create_schema()

Schema created...


In [38]:
def create_patient_financial_info_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.patient_financial_info (
                patient_id VARCHAR(50) NOT NULL,
                credit_card_number VARCHAR(30),
                credit_card_expiry VARCHAR(10),
                credit_card_cvv VARCHAR(6),
                insurance_provider VARCHAR(100),
                insurance_id VARCHAR(50),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        print('Patient financial info table created...')
    except Exception as e:
        print(f'Financial info table error: {e}')
        conn.rollback()
create_patient_financial_info_table()

Patient financial info table created...


In [ ]:
# development delay
# table data incomplete
def create_hospitals_table(conn):
    """Create hospitals table"""
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.hospitals (
                hospital_id SERIAL PRIMARY KEY,
                hospital_name VARCHAR(200) NOT NULL UNIQUE,
                address TEXT,
                city VARCHAR(100),
                state VARCHAR(50),
                zip_code VARCHAR(20),
                phone VARCHAR(30),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        print('Hospitals table created...')
    except Exception as e:
        print(f'Hospitals table error: {e}')
        conn.rollback()

In [ ]:
# development delay
# table data incomplete
def create_physicians_table(conn):
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.physicians (
                physician_id SERIAL PRIMARY KEY,
                first_name VARCHAR(100) NOT NULL,
                last_name VARCHAR(100) NOT NULL,
                specialization VARCHAR(100),
                license_number VARCHAR(50) UNIQUE,
                phone VARCHAR(30),
                email VARCHAR(255),
                active BOOLEAN DEFAULT TRUE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            );
        """)
        conn.commit()
        print('Physicians table created...')
    except Exception as e:
        print(f'Physicians table error: {e}')
        conn.rollback()

In [35]:
def create_diagnoses_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.diagnoses (
                patient_id VARCHAR(50) PRIMARY KEY,
                icd_10_code VARCHAR(10),
                diagnosis_name TEXT NOT NULL,
                diagnosis_category VARCHAR(100),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        print('Diagnoses table created...')
    except Exception as e:
        print(f'Diagnoses table error: {e}')
        conn.rollback()
create_diagnoses_table()

Diagnoses table created...


In [37]:
def create_encounters_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.encounters (
                encounter_id VARCHAR(50) PRIMARY KEY,
                patient_id VARCHAR(50) NOT NULL,
                admission_date DATE NOT NULL,
                release_date DATE,
                primary_diagnosis_code VARCHAR(10),
                died BOOLEAN DEFAULT FALSE,
                cause_of_death TEXT,
                death_date DATE,
                transferred BOOLEAN DEFAULT FALSE,
                transfer_hospitals TEXT,
                transfer_date DATE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        print('Encounters table created...')
    except Exception as e:
        print(f'Encounters table error: {e}')
        conn.rollback()
create_encounters_table()

Encounters table created...


In [28]:
def create_follow_up_appointments_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.follow_up_appointments (
                patient_id VARCHAR(50) PRIMARY KEY,
                attending_physician VARCHAR(150),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        print('Follow-up appointments table created...')
    except Exception as e:
        print(f'Follow-up appointments table error: {e}')
        conn.rollback()
create_follow_up_appointments_table()

Follow-up appointments table created...


In [30]:
def create_billing_table():
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.billing (
                patient_id VARCHAR(50) PRIMARY KEY,
                total_cost DECIMAL(10,2) NOT NULL,
                insurance_coverage DECIMAL(10,2),
                out_of_pocket DECIMAL(10,2),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        print('Billing table created...')
    except Exception as e:
        print(f'Billing table error: {e}')
        conn.rollback()
create_billing_table()
create_billing_table()

Billing table created...
Billing table created...


In [ ]:
# development delay
# table data incomplete
def create_patient_demographics_table(conn):
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.patient_demographics (
                patient_id VARCHAR(50) PRIMARY KEY,
                age INTEGER,
                marital_status VARCHAR(20),
                occupation VARCHAR(100),
                address TEXT,
                city VARCHAR(100),
                state VARCHAR(50),
                zip_code VARCHAR(20),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        conn.commit()
        print('Patient demographics table created...')
    except Exception as e:
        print(f'Patient demographics table error: {e}')
        conn.rollback()

In [ ]:
# development delay
# table data incomplete
def create_audit_log_table(conn):
    """Create audit log table for tracking changes"""
    try:
        cur = conn.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS healthcare.audit_log (
                audit_id SERIAL PRIMARY KEY,
                table_name VARCHAR(100) NOT NULL,
                record_id VARCHAR(100),
                action VARCHAR(20) NOT NULL,
                old_values JSONB,
                new_values JSONB,
                changed_by VARCHAR(100),
                changed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                
                CONSTRAINT valid_action CHECK (action IN ('INSERT', 'UPDATE', 'DELETE'))
            );
            
            CREATE INDEX IF NOT EXISTS idx_audit_table 
            ON healthcare.audit_log(table_name, changed_at);
            
            CREATE INDEX IF NOT EXISTS idx_audit_record 
            ON healthcare.audit_log(record_id);
        """)
        conn.commit()
        print('Audit log table created...')
    except Exception as e:
        print(f'Audit log table error: {e}')
        conn.rollback()

In [31]:
conn.close()